In [29]:
import pandas as pd
def save_data(coin, candle, trend, start_date, end_date, return_data=False):
    df = pd.read_csv(f'{coin}_{candle}.csv', index_col=0, parse_dates=[0])
    df.drop(columns=['market'], inplace=True)
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    
    df = df.loc[start_date:end_date]
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'datetime'}, inplace=True)
    
    df.to_csv(f'{coin}_{candle}_{trend}.csv', index=False)
    if return_data:
        return df


In [30]:
save_data('eth', 'day1', 'bulls', '2023-01-01 09:00:00', '2025-05-01 09:00:00')

In [49]:
save_data('xrp', 'day1', 'bulls', '2023-01-01 09:00:00', '2025-05-01 09:00:00')

In [15]:
import requests
import pandas as pd

def get_upbit_monthly_candles_csv_format(market='KRW-BTC', to=None, count=100):
    url = f'https://api.upbit.com/v1/candles/months'
    
    headers = {"Accept": "application/json"}
    params = {
        'market': market,
        'count': count,
    }

    if to:
        params['to'] = to  # ISO 8601 형식

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['candle_date_time_kst'])
    df = df.sort_values('datetime')  # 오름차순 정렬

    # 컬럼명 변경 및 재정렬
    df = df.rename(columns={
        'opening_price': 'open',
        'high_price': 'high',
        'low_price': 'low',
        'trade_price': 'close',
        'candle_acc_trade_volume': 'volume'
    })[['datetime', 'open', 'high', 'low', 'close', 'volume']]

    return df

# ✅ 사용 예시
df_eth_month1 = get_upbit_monthly_candles_csv_format(market='KRW-ETH', count=24)
df_xrp_month1 = get_upbit_monthly_candles_csv_format(market='KRW-XRP', count=24)

df_eth_month1.to_csv('eth_month1.csv', index=False)
df_xrp_month1.to_csv('xrp_month1.csv', index=False)

In [43]:
import numpy as np


def analyze_buy_and_hold(df,
                         start_date: str,
                         end_date: str,
                         risk_free_rate_annual: float = 0.0):
    """
    매수 후 보유 전략의 성과 지표를 계산하는 함수 (기간 지정 가능)

    Parameters
    ----------
    df : pandas.DataFrame
        최소한 다음 컬럼을 가져야 합니다:
        - 'datetime' : datetime 형식의 날짜
        - 'close'    : 해당 월 종가 (종료 시점 가격)

    start_date : str
        분석 시작일자. 예: "2023-01-01 00:00:00"
    end_date : str
        분석 종료일자. 예: "2023-365-31 23:59:59"
    risk_free_rate_annual : float, optional
        연간 무위험 이자율 (예: 0.02 는 2%). 기본값은 0 (무위험 수익률 0).

    Returns
    -------
    result : dict
        {
          'cumulative_return' : 누적 수익률 (float, 예: 0.35 → +35%),
          'max_drawdown'      : 최대 손실율 (float, 예: -0.20 → -20%),
          'sharpe_ratio'      : 연환산 샤프 지수 (float)
        }
    """

    df = df.copy()
    df['datetime'] = pd.to_datetime(df['datetime'])

    # 시작/종료일을 datetime으로 변환
    start_dt = pd.to_datetime(start_date)
    end_dt   = pd.to_datetime(end_date)

    # 기간 필터링 (start_date <= datetime <= end_date)
    mask = (df['datetime'] >= start_dt) & (df['datetime'] <= end_dt)
    df_period = df.loc[mask].sort_values('datetime').reset_index(drop=True)

    # 기간 내 데이터가 충분한지 검사 (각 달마다 시가/종가가 있어야 최소 1개의 수익률 계산 가능)
    if len(df_period) < 1:
        raise ValueError("주어진 기간에 해당하는 데이터가 없습니다. (최소 1개월 데이터 필요)")

    # ——————————————
    # 2) 월별 시가→종가 수익률 계산
    # ——————————————
    # return_i = (close_i / open_i) - 1
    df_period['return'] = df_period['close'] / df_period['open'] - 1
    returns = df_period['return']

    # 만약 1개 이하 수익률만 있다면, 샤프 계산 시 표준편차가 0이 되어 오류가 나므로
    # 이 경우에는 샤프 지수를 np.nan 처리하도록 한다.
    sufficient_for_sharpe = len(returns) >= 2

    # ——————————————
    # 3) 누적 수익률 (Cumulative Return)
    # ——————————————
    # (1 + r1) * (1 + r2) * ... * (1 + r_n) - 1
    cumulative_return = (1 + returns).prod() - 1

    # ——————————————
    # 4) 최대 손실율 (Maximum Drawdown)
    # ——————————————
    #  1) wealth_index_t = (1 + r_t).cumprod()
    #  2) running_max = wealth_index.cummax()
    #  3) drawdown_t = (wealth_index_t / running_max_t) - 1
    wealth_index = (1 + returns).cumprod()
    running_max  = wealth_index.cummax()
    drawdown     = wealth_index / running_max - 1
    max_drawdown = drawdown.min()  # 음수 값

    # ——————————————
    # 5) 샤프 지수 (Sharpe Ratio, 연환산)
    # ——————————————
    #  - 연간 무위험 이자율 → 월간 무위험 이자율 환산:
    #      rf_monthly = (1 + rf_annual)^(1/365) - 1
    rf_monthly = (1 + risk_free_rate_annual)**(1/365) - 1

    if sufficient_for_sharpe:
        # 월간 초과 수익률
        excess_returns = returns - rf_monthly

        # 평균 초과 수익률, 표준편차 (모표준편차, ddof=0)
        mean_excess = excess_returns.mean()
        std_excess  = excess_returns.std(ddof=0)

        # 표준편차가 0이 되는 경우를 대비하여, 0이면 샤프를 NaN으로 둠
        if std_excess == 0:
            sharpe_annual = np.nan
        else:
            sharpe_month  = mean_excess / std_excess
            sharpe_annual = sharpe_month * np.sqrt(365)
    else:
        sharpe_annual = np.nan

    return {
        'cumulative_return': cumulative_return,
        'max_drawdown'     : max_drawdown,
        'sharpe_ratio'     : sharpe_annual
    }

In [51]:
df = pd.read_csv('xrp_day1_bulls.csv')

start_date = "2025-02-01 09:00:00"
end_date   = "2025-04-30 09:00:00"

stats = analyze_buy_and_hold(df,
                                start_date=start_date,
                                end_date=end_date,
                                risk_free_rate_annual=0.0)

print("=== 지정 기간 Buy & Hold 전략 성과 ===")
print(f"기간               : {start_date} ~ {end_date}")
print(f"누적 수익률       : {stats['cumulative_return']*100:.2f}%")
print(f"최대 손실율       : {stats['max_drawdown']*100:.2f}%")
print(f"연환산 샤프 지수  : {stats['sharpe_ratio']:.4f}")


=== 지정 기간 Buy & Hold 전략 성과 ===
기간               : 2025-02-01 09:00:00 ~ 2025-04-30 09:00:00
누적 수익률       : -32.17%
최대 손실율       : -40.35%
연환산 샤프 지수  : -0.8824
